##### WLD SOLVER

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
     raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
2.17.0


In [4]:
import time
import math
import tensorflow.compat.v1 as tf
import numpy as np
from tensorflow.python.training.moving_averages import assign_moving_average
from scipy.stats import multivariate_normal as normal            # Generate normally distributed random numbers
from tensorflow import random_normal_initializer as norm_init    #Initializers for generating tensors with normal distributions
from tensorflow import random_uniform_initializer as unif_init   #Generating initializers with uniformly distributed tensors
from tensorflow import constant_initializer as const_init        #Generating initializers for tensors with constant values
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

class SolveAllenCahn (object):
    """ The fully-connected neural network model ."""
    def __init__ (self,sess):
        self.sess = sess
        # parameters for the PDE
        self.d = 100      # Dimensions of data
        self.T = 0.3      # Length of time for each path
        # parameters for the algorithm
        self.n_time = 20     # Composed of 20 networks
        self.n_layer = 4     # Number of layers of the neural network
        self.n_neuron = [self.d ,self.d +10 ,self.d +10, self.d ]    # Number of neurons in each layer, corresponding to input, hidden layer 1, hidden layer 2, output
        self.batch_size = 64      # Used one at a time for path calculation, 64*4=256
        self.valid_size = 256     # 256 Monte Carlo samples (paths)
        self.n_maxstep = 4000     #Iteration steps
        self.n_displaystep = 100     # Print every 100 steps
        self.learning_rate = 5e-4
        self.Yini = [0.3,0.6]       # Maximum and minimum of initial value Y0
        # some basic constants and variables
        self.h = (self.T +0.0)/self.n_time    # Interval of data in each path，delta t
        self.sqrth = math.sqrt(self.h)
        self.t_stamp = np.arange(0,self.n_time)*self.h  # Timestamp, accumulated time
        self._extra_train_ops = []  # batch moving average operation, which requires additional training of beta and gamma

    def train(self):
        # For training of neural networks
        start_time = time.time()
        # train operations create new tensorflow variable,name='global_step' with yield generator
        self.global_step = \
            tf.get_variable('global_step', [] ,
                              initializer = tf.constant_initializer(1),
                              trainable = False,dtype = tf.int32 )
        trainable_vars = tf.trainable_variables()
        grads = tf.gradients(self.loss,trainable_vars)
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        apply_op = \
            optimizer.apply_gradients(zip(grads,trainable_vars) ,
                                          global_step = self.global_step)

        train_ops = [apply_op] + self._extra_train_ops
        self.train_op = tf.group(* train_ops)

        self.loss_history = []   # Used to record loss values
        self.init_history = []   # Used to record the value of Y0
        self.run_time = []

        # for validation
        dW_valid , X_valid = self.sample_path(self.valid_size)
        feed_dict_valid = { self.dW : dW_valid,
                            self.X : X_valid,
                            self.is_training: False }
        # initialization
        step = 1
        self.sess.run (tf.global_variables_initializer())

        # Operational framework
        temp_loss = self.sess.run(self.loss ,
                                  feed_dict = feed_dict_valid )

        temp_init = self.Y0.eval()[0]
        self.loss_history.append(temp_loss)
        self.init_history.append(temp_init)
        self.run_time.append(time.time()-start_time + self.t_bd)
        print("step : %5u , loss : %.4e , " % \
                (0 ,temp_loss ) + \
                "Y0 : % .4e , runtime : %4u " % \
                (temp_init, time.time()-start_time + self.t_bd))

        # begin sgd iteration，0-4000STEP
        for _ in range (self.n_maxstep +1):
            step = self.sess.run (self.global_step)
            dW_train,X_train = self.sample_path(self.batch_size)
            self.sess.run(self.train_op,
                          feed_dict ={self.dW : dW_train ,
                                      self.X : X_train ,
                                      self.is_training : True })
            if step % self.n_displaystep == 0:   # Test the loss and the value of Y0 every 100 steps with the validation set
                temp_loss = self.sess.run(self.loss ,
                                          feed_dict = feed_dict_valid)
                temp_init = self.Y0.eval()[0]
                self.loss_history.append(temp_loss)
                self.init_history.append(temp_init )
                self.run_time.append(time.time()-start_time + self.t_bd)
                print("step : % 5u , loss : %.4e , " % \
                        ( step , temp_loss ) + \
                        " Y0 : % .4e , runtime : %4u " % \
                        (temp_init , time.time() - start_time + self.t_bd ))
            step += 1
        end_time = time.time()
        print(" running time : % .3f s " % \
                ( end_time - start_time + self.t_bd ))

    def build(self):
        # build the whole network by stacking subnetworks
        start_time = time.time ()
        # dW、X、is_training placeholder
        self.dW = tf.placeholder(tf.float32 ,[ None , self.d , self.n_time ] ,name = 'dW')   # None*100*20
        self.X = tf.placeholder(tf.float32 ,[ None , self.d , self.n_time +1] ,name = 'X')   # None*100*20
        self.is_training = tf.placeholder (tf.bool)

        # Initialize Y0\Z0
        self.Y0 = tf.Variable(tf.random_uniform([1] ,                      # u0
                                                minval = self.Yini [0] ,   # MIN0.3
                                                maxval = self.Yini [1] ,   # MAX0.6
                                                dtype = tf.float32 ));
        self.Z0 = tf.Variable (tf.random_uniform ([1,self.d] ,    # The initial value of the u-gradient, a 1*d vector
                                                minval = -.1 ,
                                                maxval =.1 ,
                                                dtype = tf.float32 ))
        self.allones = \
             tf.ones(shape = tf.stack([ tf.shape(self.dW)[0],1]) ,   # tf.shape(self.dW)[0]=len(batch),shape=(batch,1)
                         dtype = tf.float32 )

        Y = self.allones * self.Y0
        Z = tf.matmul(self.allones, self.Z0 )


        with tf.variable_scope('forward'):
            for t in range(0,self.n_time -1):

                    Y = Y - self.f_tf(self.t_stamp[t] ,
                                    self.X[:,:,t],Y,Z)* self.h
                    Y = Y + tf.reduce_sum(Z * self.dW[:,:,t],1,
                                       keep_dims = True )
                    Z = self._one_time_net(self.X[:,:,t +1] ,
                                       str(t +1))/self.d
            # terminal time
            Y = Y - self.f_tf(self.t_stamp[self.n_time -1] ,
                                  self.X[:,:,self.n_time -1] ,
                                  Y,Z)* self.h
            Y = Y + tf.reduce_sum(Z * self.dW [:,:,self.n_time -1] , 1 ,
                                      keep_dims = True )
            term_delta = Y - self.g_tf(self.T,
                                   self.X[:,:,self.n_time])
            self.clipped_delta = \
                  tf.clip_by_value(term_delta ,-50.0 , 50.0)
            self.loss = tf.reduce_mean(self.clipped_delta**2)
        self.t_bd = time.time() - start_time

    def sample_path(self, n_sample):
        # （xt,(wt-wt-1)）
        dW_sample = np.zeros([n_sample,self.d,self.n_time])
        X_sample = np.zeros([n_sample,self.d,self.n_time +1])
        for i in range(self.n_time):
            dW_sample [:,:,i] = \
               np.reshape(normal.rvs(mean = np.zeros(self.d) ,
                                     cov =1 ,
                                     size = n_sample)* self.sqrth ,
                          (n_sample,self.d))
            X_sample[:,:,i +1] = X_sample[:,:,i] + \
                                   np.sqrt(2)*dW_sample[:,:,i]
        return dW_sample, X_sample

    def f_tf(self,t,X,Y,Z ):
        # nonlinear term
        return Y - tf.pow(Y,3)

    def g_tf(self,t,X):
        # terminal conditions
        return 0.5/(1 + 0.2* tf.reduce_sum(X **2,1,keep_dims = True ))

    def _one_time_net(self , x ,name ):

        with tf.variable_scope(name):
            x_norm = self._batch_norm(x , name = 'layer0_normal')  # Normalize batch as input
            layer1 = self._one_layer(x_norm , self.n_neuron [1] ,   # Hidden Layer 1 input(batch,d),output(batch，d+10)
                                      name = 'layer1')
            layer2 = self._one_layer(layer1,self.n_neuron[2] ,  # Hidden Layer 2 input(batch,d+10),output(batch,d+10)
                                      name = 'layer2')
            z = self._one_layer(layer2 , self.n_neuron [3] , #  input(batch,d+10),output(baatch,d)
                                     activation_fn = None , name = 'final')
        return z

    def _one_layer(self , input_ , out_sz ,
                   activation_fn = tf.nn.relu ,
                   std =5.0 , name = 'linear'):

        with tf.variable_scope(name):
            shape = input_.get_shape().as_list()
            w = tf.get_variable('Matrix',
                                [shape[1], out_sz] ,tf.float32,
                                norm_init(stddev = \
                                          std / np.sqrt(shape[1]+ out_sz )))
            hidden = tf.matmul(input_ ,w )
            hidden_bn = self._batch_norm(hidden, name = 'normal')
        if activation_fn != None :
            return activation_fn(hidden_bn)
        else :
            return hidden_bn

    def _batch_norm(self , x , name ):
        """ Batch normalization """
        with tf.variable_scope(name):
            params_shape = [x.get_shape()[ -1]]   # [d,d+10,d+10,d]
            beta = tf.get_variable('beta', params_shape ,
                                         tf.float32 ,
                                         norm_init(0.0 , stddev =0.1 ,
                                         ))
            gamma = tf.get_variable( 'gamma', params_shape ,
                                         tf.float32 ,
                                         unif_init (0.1,0.5 ,
                                          ))
            mv_mean = tf.get_variable('moving_mean' ,
                                         params_shape ,
                                         tf.float32 ,
                                         const_init (0.0) ,
                                         trainable = False )
            mv_var = tf.get_variable('moving_variance' ,
                                        params_shape ,
                                        tf.float32 ,
                                        const_init(1.0) ,
                                        trainable = False )

            # These ops will only be preformed when training
            mean ,variance = tf.nn.moments(x ,[0] , name = 'moments')
            self._extra_train_ops.append (\
                 assign_moving_average(mv_mean , mean , 0.99))
            self._extra_train_ops.append (\
                 assign_moving_average(mv_var , variance , 0.99))

            mean,variance = \
                tf.cond(self.is_training ,            # control_flow_ops.cond Controls the execution flow, with the first being a condition
                                     lambda :( mean , variance ) ,
                                     lambda :( mv_mean , mv_var ))

            y = tf.nn.batch_normalization (x , mean , variance ,
                                           beta , gamma , 1e-6)

            y.set_shape( x.get_shape())
            return y

def main (name):
    tf.reset_default_graph ()
    with tf.Session() as sess :
        tf.set_random_seed (1)
        print(" Begin to solve Allen - Cahn equation ")
        model = SolveAllenCahn (sess)
        model.build()
        model.train ()
        output = np.zeros ((len(model.init_history), 4))
        output[:,0] = np.arange(len( model.init_history )) \
                             * model.n_displaystep
        output[:,1] = model.loss_history
        output[:,2] = model.init_history
        output[:,3] = model.run_time
        np.savetxt("./ AllenCahn_d100"+str(name)+".csv " ,  # Saving the output
                     output ,
                     fmt =[ '%d', '%.5e', '%.5e','%d'] ,
                     delimiter =",",
                     header ="step ,loss function , " + \
                     " target value , runtime " ,
                     comments = '')

if __name__ == '__main__':
        np.random.seed(1) # Define a random number seed
        for i in range(5):
            print(str(i)+' run:')
            main(i)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
keep_dims is deprecated, use keepdims instead


0 run:
 Begin to solve Allen - Cahn equation 
step :     0 , loss : 1.6158e-01 , Y0 :  5.4381e-01 , runtime :   18 
step :   100 , loss : 1.2692e-01 ,  Y0 :  4.9516e-01 , runtime :   52 
step :   200 , loss : 9.9422e-02 ,  Y0 :  4.5004e-01 , runtime :   82 
step :   300 , loss : 7.8032e-02 ,  Y0 :  4.0832e-01 , runtime :  109 
step :   400 , loss : 6.0837e-02 ,  Y0 :  3.6975e-01 , runtime :  139 
step :   500 , loss : 4.7267e-02 ,  Y0 :  3.3401e-01 , runtime :  168 
step :   600 , loss : 3.6248e-02 ,  Y0 :  3.0103e-01 , runtime :  196 
step :   700 , loss : 2.7612e-02 ,  Y0 :  2.7048e-01 , runtime :  226 
step :   800 , loss : 2.0776e-02 ,  Y0 :  2.4241e-01 , runtime :  255 
step :   900 , loss : 1.5424e-02 ,  Y0 :  2.1670e-01 , runtime :  284 
step :  1000 , loss : 1.1286e-02 ,  Y0 :  1.9333e-01 , runtime :  312 
step :  1100 , loss : 8.1860e-03 ,  Y0 :  1.7231e-01 , runtime :  341 
step :  1200 , loss : 5.8499e-03 ,  Y0 :  1.5347e-01 , runtime :  369 
step :  1300 , loss : 4.1310e-03